# Poker Hand Values
## Gaétan Ramet


## Import packages

In [31]:
import sys
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

## Load Data
We use pandas library to load our data. Pandas loads them into dataframes which helps us analyze our data easily. Learn more about it [here](https://www.tutorialspoint.com/python_data_science/python_pandas.htm)

In [32]:
train_data_path = "train.csv" #path where data is stored

In [33]:
train_data = pd.read_csv(train_data_path) #load data in dataframe using pandas

## Visualize data

In [34]:
train_data.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,label
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


We can see there are 11 column where first 10 column contains the cards information and the last one describing the hand it makes. 1st and 2nd column contains suit and rank of first card respectively, 3rd and 4th column suit and rank of 2nd card and so on.

## Value Computation

Let's now implement the logic to compute the value of a hand

In [35]:


def isFlush(suits):
    if len(set(suits)) == 1:
        return True
    return False

def isStraight(vals):
    #if (vals == [1, 10, 11, 12,13]) or ((vals[len(vals)-1] - vals[0] == 4) and (len(set(vals)) == 5)):
    if (vals == [1, 10, 11, 12,13]) or (vals == [1, 2, 3, 4, 5]) or (vals == [2, 3, 4, 5, 6]) or (vals == [3, 4, 5, 6, 7]) or (vals == [4,5,6,7,8]) or (vals == [5, 6, 7,8,9]) or (vals == [6, 7,8,9,10]) or (vals == [7,8,9,10,11]) or (vals == [8,9,10,11,12]) or (vals == [9,10,11,12,13]) or (vals == [1,2,11,12,13]) or (vals == [1,2,3,12,13]) or (vals == [1,2,3,4,13]):
        return True
    return False

def isFour(vals_c):
    if sorted(vals_c.values()) == [1, 4]:
        return True
    return False

def isFull(vals_c):
    if sorted(vals_c.values()) == [2, 3]:
        return True
    return False

def isSet(vals_c):
    if sorted(vals_c.values()) == [1, 1, 3]:
        return True
    return False

def isTwoPairs(vals_c):
    if sorted(vals_c.values()) == [1, 2, 2]:
        return True
    return False

def isPair(vals_c):
    if sorted(vals_c.values()) == [1, 1, 1, 2]:
        return True
    return False


def hand_value(hand):
    """Compute the value of a hand"""
    suits = [hand['S1'], hand['S2'], hand['S3'], hand['S4'], hand['S5']]
    vals = sorted([hand['C1'], hand['C2'], hand['C3'], hand['C4'], hand['C5']])
    vals_c = Counter(vals)

    is_flush = False
    if isFlush(suits):
        if isStraight(vals):
            if vals == [1, 10, 11, 12, 13]:
                # Royal Flush
                return 9
            # Straight Flush
            return 8
        # Flush
        is_flush = True

    if isFour(vals_c):
        # Four of a Kind
        return 7
    
    if isFull(vals_c):
        # Full House
        return 6
    
    if is_flush:
        # Flush
        return 5

    if isStraight(vals):
        # Straight
        return 4

    if isSet(vals_c):
        # Set
        return 3

    if isTwoPairs(vals_c):
        # Two pairs
        return 2

    if isPair(vals_c):
        return 1

    # High Card
    return 0

SyntaxError: invalid syntax (<ipython-input-35-60521d4da5f2>, line 8)

In [36]:
X_train,y_train = train_data.iloc[:,:-1], train_data.iloc[:,-1]

## Check correctness on train set


In [22]:
y_pred = X_train.apply(hand_value, axis=1)
y_pred.head()

0    0
1    1
2    1
3    1
4    0
dtype: int64

In [23]:
y_train.head()
y_train.value_counts()

0    501209
1    422498
2     47622
3     21121
4      3885
5      1996
6      1424
7       230
8        12
9         3
Name: label, dtype: int64

## Evaluate the Performance
We use the same metrics as that will be used for the test set.  
[F1 score](https://en.wikipedia.org/wiki/F1_score) are the metrics for this challenge

In [24]:
precision = precision_score(y_train,y_pred,average='micro')
recall = recall_score(y_train,y_pred,average='micro')
accuracy = accuracy_score(y_train,y_pred)
f1 = f1_score(y_train,y_pred,average='macro')

In [25]:
print("Accuracy of the model is :" ,accuracy)
print("Recall of the model is :" ,recall)
print("Precision of the model is :" ,precision)
print("F1 score of the model is :" ,f1)

Accuracy of the model is : 1.0
Recall of the model is : 1.0
Precision of the model is : 1.0
F1 score of the model is : 1.0


# Prediction on Evaluation Set

## Load Test Set
Load the test data now

In [26]:
final_test_path = "test.csv"
final_test = pd.read_csv(final_test_path)
final_test.head()
len(final_test)

25010

## Predict Test Set
Time for the moment of truth! Predict on test set and time to make the submission.

In [27]:
submission = final_test.apply(hand_value, axis=1)

## Save the prediction to csv

In [28]:
#submission = pd.DataFrame(submission)
submission.to_csv('results/submission_4.csv',header=['label'],index=False)

In [29]:
final_test_with_preds = final_test
final_test_with_preds['pred'] = submission

In [30]:
final_test_with_preds.to_csv('preds.csv', index=False)